In [18]:
import numpy as np
import scipy as sp
import plotly.express as px
from scipy import stats, optimize, interpolate

In [2]:
w0 = 5
x0 = 5
L = int(np.exp(w0*x0) / w0)
alpha = 0

**1. Function `J(x, w)`**

`J(x, w)` will evaluate `J` at `x`, `w`

$$J = |x - x_0| + \dfrac{\lambda}{1 + e^{w * x}}$$

In [3]:
def J(x, w):
    return np.abs(x-x0) + (L / (1 + np.exp(w * x)))

**2. Function `X(w)`**

`X(w)` will find the minimum `x` that solves `J` given some `w (weight)`

$$OPT = \argmin_{x} \quad J$$

In [4]:
def X(w):
    def OPT(x):
        return np.abs(x-x0) + (L / (1 + np.exp(w * x)))
    x = np.round(sp.optimize.minimize_scalar(OPT).x, 3)
    # x = sp.optimize.minimize_scalar(OPT).x
    return x

**3. Function `XCF(w)`**

`XCF(w)` is the closed form solution that we found by taking the partial derivative of `J`
$$\dfrac{\partial J}{\partial \delta} = \dfrac{\partial}{\partial \delta}\left(\delta + \dfrac{\lambda}{1 + e^{w * (x_0 + \delta)}} \right)\\\\ x = x_0 + \delta$$

In [5]:
def XCF(w):
    c = ((L*w - 2) + (np.sqrt((L*w-2)**2 - 4))) / 2
    d = (np.log(c) - w*x0) / w
    return np.round(x0+d,3)

### 1. Experiment A

The objective of this experiment is to see the effect of moving away from the predicted weight `w1` by some `alpha < w0` where `w0` is the initial weight

In [6]:
w0 = 5
x0 = 5
N = 10
d1 = {
    'w~': [np.round(w0 + (1/N)*f*w0, 2) for f in range(-(N-0),(N+1)+5)],
    'w1': [np.round(w0 + (1/N)*f*w0, 2) for f in range(-(N-0),(N+1)+5)],
    'ratio': [],
    'xs': []
}
w_max = 2*w0
L = int(np.exp(w_max*x0) / w_max)

for w1 in d1['w1']:
    d1['ratio'].append([])
    d1['xs'].append([])
    x1 = X(w1)
    for wt in d1['w~']:
        xt = X(wt)
        ratio = J(xt, w1) / J(x1, w1)
        d1['ratio'][-1].append(np.log(ratio))
        # d1['ratio'][-1].append(ratio)
        d1['xs'][-1].append(f'(xt={xt}, x1={x1})')

/var/folders/22/3f6b6q3n611gxtcqrgc0wkq80000gn/T/ipykernel_35142/1549896417.py:2: RuntimeWarning: overflow encountered in exp
  return np.abs(x-x0) + (L / (1 + np.exp(w * x)))


In [7]:
L

518470552858707230720

In [20]:
fig = px.imshow(d1['ratio'],
                labels=dict(x='w~', y='w1', color='ratio'),
                x=d1['w~'],
                y=d1['w1'],
                color_continuous_scale=["lightblue","blue","yellow","orange","red"],
                aspect='auto',
                text_auto=True)

fig.update(data=[{'customdata': np.array(d1['xs']),
    'hovertemplate': 'w~: %{x}<br>w1: %{y}<br>ratio: %{z}<br>xs: %{customdata}<extra></extra>'}])

fig.update_layout(
    title={
        'text': f'x0={x0}     w0={w0}     w_max={w_max}     lambda={L:.2e}     increments={(1/N)*w0}', 
        'x': 0.5, 
        'xanchor': 'center', 
        'yanchor': 'top'
        }
)

fig.show()

### 2. Experiment B

The objective of this experiment is to understand the `x` space with respect to `w`

In [10]:
xs = []
x0s = []
for w in d1['w1']:
    # xs.append(XCF(w))
    xs.append(X(w))
    x0s.append(x0)
    
d2 = {'w': d1['w1'], 'x': xs, 'x0': x0s}

In [11]:
fig = px.line(d2, x='w', y=['x', 'x0'])

fig.update_layout(
    title={
        'text': f'x0={x0}     w0={w0}     w_max={w_max}     lambda={L:.2e}     increments={(1/N)*w0}', 
        'x': 0.5, 
        'xanchor': 'center', 
        'yanchor': 'top'
        },
    yaxis={'title':'x'}
)
fig.show()